### Импорты

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np


from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score

### Загрузка данных

In [2]:
data = pd.read_csv("../data/singapore_airlines_reviews.csv")
data.head()

,published_date,published_platform,rating,type,text,title,helpful_votes
0,2024-03-12T14:41:14-04:00,Desktop,3,review,We used this airline to go from Singapore to L...,Ok,0
1,2024-03-11T19:39:13-04:00,Desktop,5,review,The service on Singapore Airlines Suites Class...,The service in Suites Class makes one feel lik...,0
2,2024-03-11T12:20:23-04:00,Desktop,1,review,"Booked, paid and received email confirmation f...",Don’t give them your money,0
3,2024-03-11T07:12:27-04:00,Desktop,5,review,"Best airline in the world, seats, food, servic...",Best Airline in the World,0
4,2024-03-10T05:34:18-04:00,Desktop,2,review,Premium Economy Seating on Singapore Airlines ...,Premium Economy Seating on Singapore Airlines ...,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   published_date      10000 non-null  object
 1   published_platform  10000 non-null  object
 2   rating              10000 non-null  int64 
 3   type                10000 non-null  object
 4   text                10000 non-null  object
 5   title               9999 non-null   object
 6   helpful_votes       10000 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 547.0+ KB


In [4]:
data.rating.value_counts()

rating
5    5424
4    1967
1    1057
3    1009
2     543
Name: count, dtype: int64

Заметим, что у нас присутсвует дисбаланс классов. В качестве метрики выберем balanced_accuracy, так как нас не очень сильно интересует обязательно классифицировать именно плохие отзывы или вроде того. Мы просто хотим чтобы в среднем мы отвечали довольно неплохо.

## Task 1 (easy)

In [5]:
df = data[['rating', 'text']]
corpus = df.text.to_list()

In [6]:
bow = CountVectorizer()
bow_result = bow.fit_transform(corpus).toarray()
y = df.rating
X = pd.DataFrame(bow_result, columns=bow.get_feature_names_out(), index=corpus)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [8]:
model = LogisticRegression()
model.fit(X_train, y_train)
None

/home/mitchell/different/hse-ml-course/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
y_pred_train = model.predict(X_train)
score_train = balanced_accuracy_score(y_train, y_pred_train) 
y_pred_test = model.predict(X_test)
score_test = balanced_accuracy_score(y_test, y_pred_test)
print(f"Train score: {score_train}\nTest score: {score_test}")

Train score: 0.9073782025849212
Test score: 0.4662612289177023


Получили довольно большой скор на трейне и низкий на тесте... Модель переобучилась и плохо научилась находить закономерности

## Task 2 (medium)